In [45]:
#MongoDB driver "PyMongo"
import pymongo

In [46]:
# Connect to the local server
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["samples_pokemon"]
mycol = mydb["samples_pokemon"]

In [4]:
mydoc = mycol.find({"candy_count":{"$gte":40}})

# for x in mydoc:
#     print(x)

# Q1: Creates a MongoDB database called "amazon".



In [5]:
# Connect to the local server
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
print(myclient.list_database_names())

['admin', 'config', 'crunchbase', 'enron_messages', 'local', 'mongomart_cart', 'mongomart_item', 'samples_friends', 'samples_pokemon', 'video_movieDetails', 'video_movies', 'video_reviews']


In [6]:
mydb = myclient["amazon"]

In [7]:
print(myclient.list_database_names())
# In MongoDB, a database is not created until it gets content!

['admin', 'config', 'crunchbase', 'enron_messages', 'local', 'mongomart_cart', 'mongomart_item', 'samples_friends', 'samples_pokemon', 'video_movieDetails', 'video_movies', 'video_reviews']


# Q2: Reads "reviews_electronics.16.json" and uploads each review as a separate document to the collection "reviews" in the database "amazon".



In [9]:
# import the built-in JSON library
import json

# file direction
import os

import pandas as pd

In [10]:
os.chdir(r'D:\Winter 2020\BAX 452 Machine Learning\Homework 6')

In [13]:
data = []
with io.open('reviews_electronics.16.json', encoding='utf-8') as json_data:
    for line in json_data:
        data_line = json.loads(line)
        data.append(data_line)
data[5]

{'reviewerID': 'AWO84NZ6VDYR2',
 'asin': 'B00E4KP4W6',
 'reviewerName': 'shalane rogillio',
 'helpful': [0, 0],
 'reviewText': 'I ordered two of these &#34;shatter proof&#34; protector for iPad s.  One arrived broken. Amazon refunded and ordered another one just like it.  Using the flat plastic tool to push bubbles out, the edge of this chipped.  This is definitely not up breakable and when it does break it is in big shards.  So from the 3 I have touched, only one is perfect.',
 'overall': 1.0,
 'summary': 'Not happy with this, would not recommend',
 'unixReviewTime': 1404777600,
 'reviewTime': '07 8, 2014'}

In [16]:
mydb = myclient["amazon"]
mycol = mydb["reviews"]
mycol.drop()
mycol = mydb["reviews"]
x = mycol.insert_many(data)

# Q3: Uses MongoDB's map reduce function to build a new collection "avg_scores" that averages review scores by product ("asin"). Print the first 100 entries of "avg_scores" to screen.


In [1]:
from bson.code import Code

In [28]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["amazon"]
mycol = mydb["reviews"]

In [29]:
mapper = Code("""
              function () {emit(this.asin,this.overall)}
             """)
reducer = Code("""
                function (key,value) {return Array.avg(value)}
              """)
result = mycol.map_reduce(mapper, reducer, "avg_scores")
for doc in result.find().limit(100):
    print(doc)

{'_id': '0132793040', 'value': 5.0}
{'_id': 'B00E4KP4W6', 'value': 4.545454545454546}
{'_id': 'B00E4KP8VI', 'value': 5.0}
{'_id': 'B00E4KPMC8', 'value': 2.0}
{'_id': 'B00E4KQ5C4', 'value': 5.0}
{'_id': 'B00E4KQ9GG', 'value': 3.2857142857142856}
{'_id': 'B00E4KQ9K2', 'value': 5.0}
{'_id': 'B00E4KQD4E', 'value': 4.0}
{'_id': 'B00E4KZBX8', 'value': 4.0}
{'_id': 'B00E4KZDJ0', 'value': 5.0}
{'_id': 'B00E4L35DA', 'value': 4.0}
{'_id': 'B00E4L3N9Q', 'value': 4.0}
{'_id': 'B00E4L48EA', 'value': 5.0}
{'_id': 'B00E4L7FLI', 'value': 1.0}
{'_id': 'B00E4L7TS2', 'value': 4.0}
{'_id': 'B00E4LAL82', 'value': 3.0}
{'_id': 'B00E4LBZZK', 'value': 5.0}
{'_id': 'B00E4LF2Z4', 'value': 4.333333333333333}
{'_id': 'B00E4LFP0G', 'value': 4.444444444444445}
{'_id': 'B00E4LFWWW', 'value': 4.4}
{'_id': 'B00E4LGTVU', 'value': 4.195658625514055}
{'_id': 'B00E4LGTXS', 'value': 3.923076923076923}
{'_id': 'B00E4LGVYA', 'value': 3.272727272727273}
{'_id': 'B00E4LGWLW', 'value': 3.5}
{'_id': 'B00E4LGXL6', 'value': 5.0}
{

# Q4: Uses MongoDB's map reduce function to build a new collection "weighted_avg_scores" that averages review scores by product ("asin"), weighted by the number of helpful votes (The base weight is 1 and for every additional helpful vote add 1 to weight. e.g. a "[3, 5]" value on "helpful" column should use 3 + 1 = 4 as weight, 3 being the additional votes and 1 being the base weight). Print the first 100 entries of "weighted_avg_scores" to screen.



In [47]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["amazon"]
mycol = mydb["reviews"]

In [48]:
mapper = Code("""
    function() {       
        var key = this.asin;
        var value = {
            base: this.helpful[0]+1,
            rate: this.overall*(this.helpful[0]+1),
            test: (this.overall*(this.helpful[0]+1))/(this.helpful[0]+1)};
        emit(key, value);};
             """)

In [49]:
reducer = Code("""
    function(key, value) {
        reduce = { rate: value[0].rate, base: value[0].base,test:(value[0].rate/value[0].base) };   
        for (var idx = 1; idx < value.length; idx++) {
            reduce.rate += value[idx].rate;
            reduce.base += value[idx].base;};
        reduce.test = (reduce.rate/reduce.base);
        return reduce;};
              """)

In [50]:
finalizer = Code("""function (key, reducedVal) {
    reducedVal.avg = reducedVal.rate/reducedVal.base;
    return reducedVal;
                    };""")

In [51]:
result = mycol.map_reduce(mapper, reducer, "test1")


In [52]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["amazon"]
mycol = mydb["test1"]
for i in mycol.find().limit(10):
    print(i)

{'_id': '0132793040', 'value': {'base': 2.0, 'rate': 10.0, 'test': 5.0}}
{'_id': 'B00E4KP4W6', 'value': {'rate': 89.0, 'base': 19.0, 'test': 4.684210526315789}}
{'_id': 'B00E4KP8VI', 'value': {'base': 1.0, 'rate': 5.0, 'test': 5.0}}
{'_id': 'B00E4KPMC8', 'value': {'rate': 12.0, 'base': 6.0, 'test': 2.0}}
{'_id': 'B00E4KQ5C4', 'value': {'base': 2.0, 'rate': 10.0, 'test': 5.0}}
{'_id': 'B00E4KQ9GG', 'value': {'rate': 59.0, 'base': 16.0, 'test': 3.6875}}
{'_id': 'B00E4KQ9K2', 'value': {'rate': 65.0, 'base': 13.0, 'test': 5.0}}
{'_id': 'B00E4KQD4E', 'value': {'base': 1.0, 'rate': 4.0, 'test': 4.0}}
{'_id': 'B00E4KZBX8', 'value': {'base': 2.0, 'rate': 8.0, 'test': 4.0}}
{'_id': 'B00E4KZDJ0', 'value': {'base': 1.0, 'rate': 5.0, 'test': 5.0}}
